In [12]:
import cudf
import pandas as pd
import numpy as np
IS_TRAIN = True
IS_SUBMIT = False

In [13]:
if IS_TRAIN:
    candidates_3 = cudf.read_parquet('/root/autodl-tmp/ottodata/tmp/co4train_v2.parquet')
    candidates_1 = cudf.read_parquet('/root/autodl-tmp/ottodata/tmp/pe4train.parquet')
    candidates_2 = cudf.read_parquet('/root/autodl-tmp/ottodata/tmp/bm4train.parquet')
else:
    candidates_1 = cudf.read_parquet('/root/autodl-tmp/ottodata/tmp/pe4test.parquet')
    candidates_3 = cudf.read_parquet('/root/autodl-tmp/ottodata/tmp/co4test2.parquet')
    # candidates_2 = cudf.read_parquet('/root/autodl-tmp/ottodata/tmp/pe4test.parquet')

In [14]:
candidates_1 = candidates_1.to_pandas()
candidates_2 = candidates_2.to_pandas()
candidates_3 = candidates_3.to_pandas()

In [15]:
candidates_2 = candidates_2.reset_index(drop=True)

In [16]:
candidates_1['labels_2'] = candidates_2['labels']
candidates_1['labels_3'] = candidates_3['labels']
# candidates_1 = candidates_1.to_pandas()
candidates_1['labels'] = candidates_1.apply(lambda x:np.unique(np.concatenate([x['labels'],x['labels_2'],x['labels_3']])),axis=1)

In [17]:
pred_df = candidates_1[['session', 'type', 'labels']]

In [18]:
if IS_SUBMIT:
    submit = pred_df.copy()
    submit['labels'] = submit['labels'].apply(lambda x: x[:20])
    submit['labels'] = submit['labels'].apply(lambda x: ' '.join([str(l) for l in x]))
    submit['session'] = submit['session'].astype('str') + '_' + submit['type'].astype('str')
    submit = submit[['session', 'labels']]
    submit.rename(columns={'session': 'session_type'}, inplace=True)
    submit.to_csv('submission.csv.gz', index=False)

In [19]:
if IS_TRAIN:
# COMPUTE METRIC
    test_labels = pd.read_parquet('/root/autodl-tmp/ottodata/valid/test_labels.parquet')
    score = 0
    weights = {'clicks': 0.10, 'carts': 0.30, 'orders': 0.60}
    for t in ['clicks','carts','orders']:
        sub = pred_df.loc[pred_df['type']==t].copy()
        test_labels_sub = test_labels.loc[test_labels['type']==t]
        test_labels_sub = test_labels_sub.merge(sub, how='left', on=['session'])
        test_labels_sub['hits'] = test_labels_sub.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
        test_labels_sub['gt_count'] = test_labels_sub.ground_truth.str.len().clip(0,20)
        recall = test_labels_sub['hits'].sum() / test_labels_sub['gt_count'].sum()
        score += weights[t]*recall
        print(f'{t} recall =',recall)
    print('=============')
    print('Overall Recall =',score)
    print('=============')

clicks recall = 0.6535509992970799
carts recall = 0.5179450529244625
orders recall = 0.7121955423344175
Overall Recall = 0.6480559412076972


In [20]:
# if IS_TRAIN:
#     pred_df.to_parquet('/root/autodl-tmp/ottodata/tmp/itemcf4train.parquet')
# else:
#     pred_df.to_parquet('/root/autodl-tmp/ottodata/tmp/itemcf4test.parquet')

In [21]:
pred_df['labels'].apply(lambda x: len(x)).mean()

101.98727273433852